# Notebook 1: EDA

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
import numpy as np
from azureml.core import Workspace, Dataset
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
from datetime import datetime, timedelta
import matplotlib as mpl
import matplotlib.pyplot as plt

1. Forest Fire Dataset

In [3]:
subscription_id = 'f0458613-2441-4d07-bf43-8d17d9e90a56'
resource_group = 'capstone-project'
workspace_name = 'ML_forestfireprediction'
workspace = Workspace(subscription_id, resource_group, workspace_name)



In [ ]:
subscription_id = 'f0458613-2441-4d07-bf43-8d17d9e90a56'
resource_group = 'capstone-project'
workspace_name = 'ML_forestfireprediction'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='forestfire')
forest_fire_dataset=dataset.to_pandas_dataframe()

Message: rslex failed, falling back to clex.
Payload: {"pid": 5659, "source": "azureml.dataprep", "version": "4.11.3", "trace": "azureml|data|tabular_dataset.py, line 169 in function <lambda>.\nazureml|data|dataset_error_handling.py, line 107 in function _try_execute.\nazureml|data|tabular_dataset.py, line 169 in function to_pandas_dataframe.", "subscription": "", "run_id": "", "resource_group": "", "workspace_name": "", "experiment_id": "", "location": "", "rslex_version": "2.18.3"}


In [11]:
forest_fire_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25210 entries, 0 to 25209
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ID          25210 non-null  int64         
 1   FID         25210 non-null  int64         
 2   SRC_AGENCY  25210 non-null  object        
 3   FIRE_ID     25210 non-null  object        
 4   FIRENAME    25210 non-null  object        
 5   LATITUDE    25210 non-null  float64       
 6   LONGITUDE   25210 non-null  float64       
 7   YEAR        25210 non-null  int64         
 8   MONTH       25210 non-null  int64         
 9   DAY         25210 non-null  int64         
 10  REP_DATE    25210 non-null  datetime64[ns]
 11  ATTK_DATE   0 non-null      object        
 12  OUT_DATE    24720 non-null  datetime64[ns]
 13  DECADE      25210 non-null  object        
 14  SIZE_HA     25210 non-null  object        
 15  CAUSE       25210 non-null  object        
 16  PROTZONE    25210 non-

In [12]:
forest_fire_dataset['OUT_DATE'] = forest_fire_dataset['OUT_DATE'].fillna(forest_fire_dataset['REP_DATE'])
forest_fire_dataset['SIZE_HA']=forest_fire_dataset['SIZE_HA'].astype(str).astype(float)

In [13]:
fig = px.scatter(forest_fire_dataset, x="REP_DATE", y="SIZE_HA", color='CAUSE',size="SIZE_HA")

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()


In [14]:
forest_fire_dataset.head()

,ID,FID,SRC_AGENCY,FIRE_ID,FIRENAME,LATITUDE,LONGITUDE,YEAR,MONTH,DAY,REP_DATE,ATTK_DATE,OUT_DATE,DECADE,SIZE_HA,CAUSE,PROTZONE,FIRE_TYPE,MORE_INFO,CFS_REF_ID,CFS_NOTE1,CFS_NOTE2,ACQ_DATE,SRC_AGY2,ECOZONE,ECOZ_REF,ECOZ_NAME,ECOZ_NOM
0,1,224484,SK,GLASS08,GLASS08,53.182254,-105.844939,1989,7,8,1989-07-08,None,1989-07-08,1980-1989,0.01,H,,Wildfire,,SK-1989-GLASS08,,,2020-04-03,SK,9,9,Boreal PLain,Plaines bornales
1,2,224485,SK,MACDOWALL2,MACDOWALL2,53.005200,-106.010400,1987,6,20,1987-06-20,None,1987-06-20,1980-1989,0.50,H,,Wildfire,,SK-1987-MACDOWALL2,,,2020-04-03,SK,9,9,Boreal PLain,Plaines bornales
2,3,224486,SK,07PA-JSIR4,07PA-JSIR4,53.071200,-104.906000,2007,5,8,2007-05-08,None,2007-05-09,2000-2009,80.00,H,,Wildfire,,SK-2007-07PA-JSIR4,,,2020-04-03,SK,9,9,Boreal PLain,Plaines bornales
3,4,224487,SK,GLASS11,GLASS11,53.182300,-105.843800,1990,7,8,1990-07-08,None,1990-07-08,1990-1999,0.01,H,,Wildfire,,SK-1990-GLASS11,,,2020-04-03,SK,9,9,Boreal PLain,Plaines bornales
4,5,224488,SK,SWEATLODGE,SWEATLODGE,53.109400,-105.906300,1995,5,30,1995-05-30,None,1995-05-30,1990-1999,1.20,H,,Wildfire,,SK-1995-SWEATLODGE,,,2020-04-03,SK,9,9,Boreal PLain,Plaines bornales


In [15]:
forest_fire_dataset.to_csv('forest_fire.csv', encoding='utf-8', index=False)

##### 2. Weather Station Dataset

###### 2.1. Weather Station Raw Data

In [4]:

import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get("WeatherStationRawData", version="3")

#df = pd.read_csv(data_asset.path)
df.head(20)

Found the config file in: /config.json


,x,y,STN_ID,STATION_NAME,PROV_STATE_TERR_CODE,ENG_PROV_NAME,FRE_PROV_NAME,COUNTRY,LATITUDE,LONGITUDE,TIMEZONE,ELEVATION,CLIMATE_IDENTIFIER,TC_IDENTIFIER,WMO_IDENTIFIER,STATION_TYPE,NORMAL_CODE,PUBLICATION_CODE,DISPLAY_CODE,ENG_STN_OPERATOR_ACRONYM,FRE_STN_OPERATOR_ACRONYM,ENG_STN_OPERATOR_NAME,FRE_STN_OPERATOR_NAME,FIRST_DATE,LAST_DATE,HLY_FIRST_DATE,HLY_LAST_DATE,DLY_FIRST_DATE,DLY_LAST_DATE,MLY_FIRST_DATE,MLY_LAST_DATE,HAS_MONTHLY_SUMMARY,HAS_NORMALS_DATA,HAS_HOURLY_DATA
0,-66.116667,48.133333,8496,CARLETON SUR MER,QC,QUEBEC,QUÉBEC,CAN,480800000,-660700000,EST,541.0,705AA86,NaN,NaN,NaN,NaN,1.0,9.0,NaN,NaN,NaN,NaN,1968-10-01 00:00:00,1968-10-31 00:00:00,NaN,NaN,1968-10-01 00:00:00,1968-10-31 00:00:00,NaN,NaN,Y,N,N
1,-81.483333,45.966667,8997,KILLARNEY (AUT),ON,ONTARIO,ONTARIO,CAN,455800000,-812900000,EST,196.3,605DJ25,WBE,71460.0,Climate-Auto,NaN,NaN,NaN,ECCC - MSC,ECCC - SMC,Environment and Climate Change Canada - Meteor...,Environnement et Changement climatique Canada ...,1992-12-02 00:00:00,2023-02-24 05:00:00,1994-02-01 02:00:00,2023-02-24 05:00:00,1992-12-02 00:00:00,2023-02-24 00:00:00,NaN,NaN,Y,N,Y
2,-70.003611,60.023056,10205,KANGIRSUK A,QC,QUEBEC,QUÉBEC,CAN,600123000,-700013000,EST,123.8,7113382,YAS,NaN,NaN,NaN,NaN,NaN,DND,MDN,Department of National Defence,Ministère de la Défense nationale,1992-10-21 07:00:00,2015-09-10 13:00:00,1992-10-21 07:00:00,2015-09-10 13:00:00,NaN,NaN,NaN,NaN,N,N,Y
3,-68.422222,58.089444,52179,KUUJJUAQ A,QC,QUEBEC,QUÉBEC,CAN,580522000,-682520000,EST,39.9,7113535,YVP,71906.0,Aviation-Staffed,NaN,NaN,NaN,NAVCAN,NAVCAN,NAV Canada,NAV Canada,2014-03-04 10:00:00,2023-11-07 12:30:01,2014-03-04 10:00:00,2023-11-07 12:30:01,2014-03-06 00:00:00,2023-11-07 00:00:00,NaN,NaN,Y,N,Y
4,-79.633333,45.250000,4506,ROSSEAU LAKE,ON,ONTARIO,ONTARIO,CAN,451500000,-793800000,EST,243.8,6117189,NaN,NaN,NaN,NaN,1.0,9.0,NaN,NaN,NaN,NaN,1968-12-01 00:00:00,1969-12-31 00:00:00,NaN,NaN,1968-12-01 00:00:00,1969-12-31 00:00:00,1969-01-01 00:00:00,1969-12-01 00:00:00,Y,N,N
5,-80.083333,44.366667,4513,STAYNER,ON,ONTARIO,ONTARIO,CAN,442200000,-800500000,EST,292.6,6117998,NaN,NaN,NaN,NaN,1.0,9.0,NaN,NaN,NaN,NaN,1969-01-01 00:00:00,1970-12-01 00:00:00,NaN,NaN,1969-05-01 00:00:00,1970-10-31 00:00:00,1969-01-01 00:00:00,1970-12-01 00:00:00,Y,N,N
6,-79.816667,44.033333,4521,TOTTENHAM,ON,ONTARIO,ONTARIO,CAN,440200000,-794900000,EST,250.9,6118848,NaN,NaN,NaN,G,1.0,8.0,NaN,NaN,NaN,NaN,1971-01-01 00:00:00,1979-12-01 00:00:00,NaN,NaN,1971-09-01 00:00:00,1979-11-30 00:00:00,1971-01-01 00:00:00,1979-12-01 00:00:00,Y,N,N
7,-81.883333,42.866667,4541,ALVINSTON,ON,ONTARIO,ONTARIO,CAN,425200000,-815300000,EST,221.0,6120250,NaN,NaN,NaN,NaN,1.0,9.0,NaN,NaN,NaN,NaN,1965-01-01 00:00:00,1969-12-31 00:00:00,NaN,NaN,1965-06-01 00:00:00,1969-12-31 00:00:00,1965-01-01 00:00:00,1969-12-01 00:00:00,Y,N,N
8,-81.916667,43.033333,4542,ARKONA,ON,ONTARIO,ONTARIO,CAN,430200000,-815500000,EST,182.9,6120315,NaN,NaN,NaN,NaN,1.0,9.0,NaN,NaN,NaN,NaN,1882-01-01 00:00:00,1915-12-01 00:00:00,NaN,NaN,1882-07-01 00:00:00,1915-03-31 00:00:00,1882-01-01 00:00:00,1915-12-01 00:00:00,Y,N,N
9,-70.883333,45.600000,5386,LAKE MEGANTIC,QC,QUEBEC,QUÉBEC,CAN,453600000,-705300000,EST,400.5,7023800,NaN,NaN,NaN,NaN,1.0,9.0,NaN,NaN,NaN,NaN,1913-01-01 00:00:00,1947-12-01 00:00:00,NaN,NaN,1913-10-01 00:00:00,1947-07-31 00:00:00,1913-01-01 00:00:00,1947-12-01 00:00:00,Y,N,N


##### 2.2. Weather Station Fire POint SQL View

In [4]:
dataset = Dataset.get_by_name(workspace, name='WeatherStationInfo')
weather_station=dataset.to_pandas_dataframe()

Engine process terminated with returncode=-2
RuntimeError('Engine process terminated. Please try running again. |session_id=l_5cef4aef-229f-4e82-9dd8-4a31830a8145')


KeyboardInterrupt: 

In [17]:
weather_station.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726 entries, 0 to 725
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ID                    726 non-null    int64         
 1   STN_ID                726 non-null    object        
 2   STATION_NAME          726 non-null    object        
 3   PROV_STATE_TERR_CODE  726 non-null    object        
 4   LATITUDE              726 non-null    float64       
 5   LONGITUDE             726 non-null    float64       
 6   TIMEZONE              724 non-null    object        
 7   CLIMATE_IDENTIFIER    726 non-null    object        
 8   FIRST_DATE            726 non-null    datetime64[ns]
 9   LAST_DATE             726 non-null    datetime64[ns]
 10  DLY_FIRST_DATE        726 non-null    datetime64[ns]
 11  DLY_LAST_DATE         726 non-null    datetime64[ns]
dtypes: datetime64[ns](4), float64(2), int64(1), object(5)
memory usage: 68.2+ KB


In [18]:
weather_station.head()

,ID,STN_ID,STATION_NAME,PROV_STATE_TERR_CODE,LATITUDE,LONGITUDE,TIMEZONE,CLIMATE_IDENTIFIER,FIRST_DATE,LAST_DATE,DLY_FIRST_DATE,DLY_LAST_DATE
0,10005,3124,MACRORIE,SK,51.316667,-107.083333,CST,4024880,1955-01-01,1958-12-01 00:00:00,1955-01-01,1958-11-30
1,10006,3127,MAPLE CREEK CDA EPF,SK,49.816667,-109.666667,CST,4024937,1954-01-01,1961-12-01 00:00:00,1954-01-01,1961-08-31
2,10007,3137,PORTREEVE,SK,50.883333,-109.050000,CST,4026175,1981-01-01,2002-10-31 00:00:00,1981-08-01,2002-10-31
3,10008,3155,SWIFT CURRENT A,SK,50.300000,-107.683333,CST,4028040,1938-01-01,2011-06-30 12:00:00,1938-05-01,2011-06-30
4,10009,3162,WOODROW,SK,49.566667,-106.783333,CST,4029030,1962-01-01,2008-01-31 00:00:00,1962-04-01,2008-01-31


##### 3. Weather Dataset

In [7]:

dataset = Dataset.get_by_name(workspace, name='weatherdata')
weather_df=dataset.to_pandas_dataframe()

In [24]:
weather_df.head()

,LONGITUDE,LATITUDE,MEAN_TEMPERATURE,STATION_NAME,LOCAL_DAY,MAX_REL_HUMIDITY,LOCAL_DATE,COOLING_DEGREE_DAYS,MIN_REL_HUMIDITY,TOTAL_PRECIPITATION,MAX_TEMPERATURE,SNOW_ON_GROUND,LOCAL_MONTH,TOTAL_SNOW,HEATING_DEGREE_DAYS,LOCAL_YEAR,DIRECTION_MAX_GUST,MIN_TEMPERATURE,SPEED_MAX_GUST,TOTAL_RAIN,CLIMATE_IDENTIFIER
0,-101.866666,52.083333,-16.8,PELLY,30,None,2003-01-30,0,None,1,-11.5,37,1,1,34.8,2003,None,-22,None,0,4086000
1,-101.866666,52.083333,-11.3,PELLY,31,None,2003-01-31,0,None,0,-5.5,38,1,0,29.3,2003,None,-17,None,0,4086000
2,-101.866666,52.083333,-11.5,PELLY,1,None,2003-02-01,0,None,2.6,-9.5,38,2,2.6,29.5,2003,None,-13.5,None,0,4086000
3,-101.866666,52.083333,-12.5,PELLY,2,None,2003-02-02,0,None,0,-11,39,2,0,30.5,2003,None,-14,None,0,4086000
4,-101.866666,52.083333,-18,PELLY,3,None,2003-02-03,0,None,0,-14.5,39,2,0,36,2003,None,-21.5,None,0,4086000


In [15]:
percent_missing = weather_df.isnull().sum() * 100 / len(weather_df)
#sns.barplot(percent_missing)
round_nums = round(percent_missing,1)
fig = px.bar(round_nums,
             text_auto = True)


fig.update_layout(
    autosize=False,
    width=800,
    height=800,
)
fig.show()

#### 4. Weather Forest Fire

In [7]:
dataset = Dataset.get_by_name(workspace, name='weatherforestfire')
forest_fire_station_point=dataset.to_pandas_dataframe()

Message: rslex failed, falling back to clex.
Payload: {"pid": 4402, "source": "azureml.dataprep", "version": "4.11.3", "trace": "azureml|data|tabular_dataset.py, line 169 in function <lambda>.\nazureml|data|dataset_error_handling.py, line 107 in function _try_execute.\nazureml|data|tabular_dataset.py, line 169 in function to_pandas_dataframe.", "subscription": "", "run_id": "", "resource_group": "", "workspace_name": "", "experiment_id": "", "location": "", "rslex_version": "2.18.3"}
thread 'tokio-runtime-worker' panicked at 'send should succeed: SendError { .. }', /src/dataprep/Core/rust_lex/rslex-http-stream/src/http_client/hyper_client/execution.rs:78:37
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


In [8]:
forest_fire_station_point.head()

,FID,FIRE_ID,FOREST_FIRE_LATITUDE,FOREST_FIRE_LONGITUDE,REP_DATE,OUT_DATE,SIZE_HA,YEAR,MONTH,DAY,CLIMATE_IDENTIFIER,WEATHER_STATION_LATITUDE,WEATHER_STATION_LONGITUDE,DLY_FIRST_DATE,DLY_LAST_DATE
0,224484,GLASS08,53.182254,-105.844939,1989-07-08,1989-07-08,0.01,1989,7,8,4024880,51.316667,-107.083333,1955-01-01,1958-11-30
1,224485,MACDOWALL2,53.005200,-106.010400,1987-06-20,1987-06-20,0.5,1987,6,20,4024880,51.316667,-107.083333,1955-01-01,1958-11-30
2,224486,07PA-JSIR4,53.071200,-104.906000,2007-05-08,2007-05-09,80,2007,5,8,4024880,51.316667,-107.083333,1955-01-01,1958-11-30
3,224487,GLASS11,53.182300,-105.843800,1990-07-08,1990-07-08,0.01,1990,7,8,4024880,51.316667,-107.083333,1955-01-01,1958-11-30
4,224488,SWEATLODGE,53.109400,-105.906300,1995-05-30,1995-05-30,1.2,1995,5,30,4024880,51.316667,-107.083333,1955-01-01,1958-11-30
